In [ ]:
import cobra
from cobra import flux_analysis
import pandas as pd

iBsu1147 = cobra.io.read_sbml_model("C:/Users/sofie/OneDrive/Documents/Master - Data/ModelSuc.sbml")    #model with new BOF
iBsu1147og = cobra.io.read_sbml_model("C:/Users/sofie/OneDrive/Documents/Master - Data/OGModelSuc.sbml") 

In [ ]:
#FVA new BOF model:
FVAnew = flux_analysis.flux_variability_analysis(iBsu1147, fraction_of_optimum= 0.95)

In [ ]:
#FVA original BOF model:
FVAog = flux_analysis.flux_variability_analysis(iBsu1147og, fraction_of_optimum= 0.95)

In [ ]:
#Checking to see if i can remove the biomass synthesis reactions so that i can make a plot with the FVA results.
Reactions_remove_new = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]

filtered_new = pd.DataFrame(columns=FVAnew.columns)
removed_new = pd.DataFrame(columns=FVAnew.columns)

#I want to loop trough the reactions that are indexed in the FVA dataframe and if that idex is one of the reactions in Reactions_remove list, then this reaction
#will be reomved from the filtered dataframe. I also made another dataframe for the FVA results of the removed reactions, because it is interesting to get to see
#those results as well.
for reaction_id in FVAnew.index:
    # Check if the reaction ID is in the Reactions_remove list
    if reaction_id in Reactions_remove_new:
        # Append the reaction to the removed_df DataFrame
        removed_new.loc[reaction_id] = FVAnew.loc[reaction_id]
    else:
        # Append the reaction to the filtered_df DataFrame
        filtered_new.loc[reaction_id] = FVAnew.loc[reaction_id]


print(len(filtered_new))


#Doing the same for the model with the original BOF:
Reactions_remove_old = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006"]

filtered_OG = pd.DataFrame(columns=FVAog.columns)
removed_OG = pd.DataFrame(columns=FVAog.columns)

#I want to loop trough the reactions that are indexed in the FVA dataframe and if that idex is one of the reactions in Reactions_remove list, then this reaction
#will be reomved from the filtered dataframe. I also made another dataframe for the FVA results of the removed reactions, because it is interesting to get to see
#those results as well.
for reaction_id in FVAog.index:
    # Check if the reaction ID is in the Reactions_remove list
    if reaction_id in Reactions_remove_old:
        # Append the reaction to the removed_df DataFrame
        removed_OG.loc[reaction_id] = FVAog.loc[reaction_id]
    else:
        # Append the reaction to the filtered_df DataFrame
        filtered_OG.loc[reaction_id] = FVAog.loc[reaction_id]

# Print the filtered DataFrame
print(len(filtered_OG))

In [ ]:
"""max_range_reaction_id = 
max_range_value = float('-inf')

# Iterate through the reaction IDs and flux ranges
for reaction_id, flux_range in zip(iBsu1147.reactions, FVAnew_range):
    if flux_range is not None:  # Skip reactions with missing flux ranges
        if flux_range > max_range_value:
            max_range_value = flux_range
            max_range_reaction_id = reaction_id

print("Reaction ID with the biggest flux range:", max_range_reaction_id)
print("Flux range:", max_range_value)"""

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


#Making lists of the mid-values and the range values for the newBOF model and the origial model with the same fluxes.
FVAnew_mid = []
for index, row in filtered_new.iterrows():
    mid_value = (row['minimum'] + row['maximum']) / 2
    FVAnew_mid.append(mid_value)

FVAnew_range =[]
for index, row in filtered_new.iterrows():
    range_value = abs(row["maximum"] - row["minimum"])
    FVAnew_range.append(range_value)

FVAog_mid = []
for index, row in filtered_OG.iterrows():
    mid_value = (row['minimum'] + row['maximum']) / 2
    FVAog_mid.append(mid_value)

FVAog_range =[]
for index, row in filtered_OG.iterrows():
    range_valueog = abs(row["maximum"] - row["minimum"])
    FVAog_range.append(range_valueog)

In [ ]:
# Merge the dataframes based on reaction IDs
merged_df = pd.merge(filtered_new, filtered_OG, left_index=True, right_index=True, how='outer')

min_diff = pd.DataFrame(columns=merged_df.columns)
max_diff = pd.DataFrame(columns=merged_df.columns)

# Set the column names for comparison
min_new = 'minimum_x'  # Replace with the actual column name from df1
min_og = 'minimum_y'  # Replace with the actual column name from df2
max_new = 'maximum_x'  # Replace with the actual column name from df1
max_og = 'maximum_y'  # Replace with the actual column name from df2

# Set the threshold value
threshold = 0.5  # Adjust this value as needed

# Iterate through the merged dataframe
for index, row in merged_df.iterrows():
    # Compare the minimum and maximum values and check if the difference exceeds the threshold
    if abs(row[min_new] - row[min_og]) > threshold:
        min_diff = min_diff.append(row)
    if abs(row[max_new] - row[max_og]) > threshold:
        max_diff = max_diff.append(row)

# Print the filtered dataframes
print("Rows where the difference in minimum values exceeds the threshold:")
print(min_diff)

print("Rows where the difference in maximum values exceeds the threshold:")
print(max_diff)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

#Colors = ['#1f78b4', '#b2df8a', '#33a02c', '#cab2d6', '#a2438d']

# Create a figure and axes for range values
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6), sharex=True)


# Create a list of reaction numbers
reaction_numbers = range(1, len(filtered_OG) + 1)

# Plot the range values for model 1
ax1.scatter(reaction_numbers, FVAnew_range, s= 20, color='#a2438d', label='new BOF')

# Plot the range values for model 2
ax1.scatter(reaction_numbers, FVAog_range, s= 20, color='#8E8E8E', label='Original BOF', alpha= 0.5)

# Set labels for the range subplot
ax1.set_ylabel('Log(Range)', fontsize = 12)   #Log(Range)
ax1.grid(axis='y', linestyle='--', alpha=0.5)
#ax1.set_ylim(0.0, 10)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.set_yscale('log')
#ax1.text(0.01, 0.95, 'A', transform=ax1.transAxes, fontsize=16, fontweight='bold')

# Plot the mid-values for model 1
ax2.scatter(reaction_numbers, FVAnew_mid, s=20, c='#a2438d', marker='o', label='new BOF')

# Plot the mid-values for model 2
ax2.scatter(reaction_numbers, FVAog_mid, s=20, c='#8E8E8E', marker='o', label='Original BOF', alpha = 0.5)

# Set y-axis scale to logarithmic for mid-value subplot
ax2.set_yscale('log')

# Set labels for the mid-value subplot
ax2.set_xlabel('Reaction number', fontsize = 12) #Reaction number
ax2.set_ylabel('Log(Mid-Value)', fontsize = 12)   #Log(Mid-Value)
#ax2.set_ylim(0.0000001, )
ax2.grid(axis='y', linestyle='--', alpha=0.5)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
#ax2.text(0.01, 0.95, 'B', transform=ax2.transAxes, fontsize=16, fontweight='bold')


# Set the title and legend for the subplots
ax1.legend(fontsize = 10, loc="lower left")
ax2.legend(fontsize = 10, loc="lower left")

# Adjust spacing between subplots
plt.tight_layout()
ax1.tick_params(axis='both', labelsize=11)
ax2.tick_params(axis='both', labelsize=11)
# Show the plot
plt.savefig('C:/Users/sofie/OneDrive/Documents/Master - Data/Figurer/FVA Succinate.svg', dpi= 300)
plt.show()